# 爬取雪球网热股

通过浏览器F12工具分析后台请求的数据

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=10&type=10  全球1小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=10&type=20  全球24小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=12&type=12  沪深1小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=12&type=22  沪深24小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=13&type=13  港股1小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=13&type=23  港股24小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=11&type=11  美股1小时最热

https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=11&type=21  美股24小时最热

In [1]:
import requests
import json
import pandas as pd

In [2]:
# 获取cookie
headers = {'user-agent': 'Mozilla/5.0'}
ck = requests.get("https://xueqiu.com/", headers=headers).cookies
ck

<RequestsCookieJar[Cookie(version=0, name='xq_a_token', value='17067303557fc0af0961063ffb2aa2341c3132a4', port=None, port_specified=False, domain='.xueqiu.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'httponly': None}, rfc2109=False), Cookie(version=0, name='xq_r_token', value='c1476ba66a6a12fbe62ab833a29e4445bc84385e', port=None, port_specified=False, domain='.xueqiu.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'httponly': None}, rfc2109=False), Cookie(version=0, name='xqat', value='17067303557fc0af0961063ffb2aa2341c3132a4', port=None, port_specified=False, domain='.xueqiu.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'httponly': No

In [3]:
def get_data(url):
    r = requests.get(url, headers=headers, cookies=ck)
    for stock_info in json.loads(r.text)['data']['items']:
        yield [stock_info['name'], float(stock_info['value'])]


result = pd.DataFrame(columns=pd.MultiIndex.from_product([['全球', '沪深', '港股', '美股'], 
                                                          ['1小时内最热', '24小时内最热'],
                                                          ['名称', '热度']]))

urls = ["https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=10&type=10",  # 全球1小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=10&type=20",  # 全球24小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=12&type=12",  # 沪深1小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=12&type=22",  # 沪深24小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=13&type=13", # 港股1小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=13&type=23", # 港股24小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=11&type=11", # 美股1小时最热
        "https://stock.xueqiu.com/v5/stock/hot_stock/list.json?size=8&_type=11&type=21"  # 美股24小时最热
       ]

for idx, url in enumerate(urls):
    result.iloc[:, 2*idx:(2*idx+2)] = list(get_data(url))
result.index += 1

In [4]:
result

全球                              沪深                           \
         1小时内最热         24小时内最热          1小时内最热         24小时内最热            
             名称      热度      名称       热度     名称      热度      名称       热度   
1          大族激光  2447.0    涪陵榨菜  46749.0   大族激光  2447.0    涪陵榨菜  46749.0   
2           拼多多  2121.0    大族激光  41026.0   涪陵榨菜  1760.0    大族激光  41026.0   
3          涪陵榨菜  1760.0    申通快递  32163.0   光线传媒  1535.0    申通快递  32163.0   
4  油气开采ETF-SPDR  1565.0    格力电器  32058.0   格力电器  1410.0    格力电器  32058.0   
5            苹果  1545.0     沃尔德  26723.0   恒顺醋业  1271.0     沃尔德  26723.0   
6          光线传媒  1535.0    中国平安  24678.0    欧菲光  1260.0    中国平安  24678.0   
7          格力电器  1410.0    天齐锂业  23201.0   东阿阿胶  1068.0    天齐锂业  23201.0   
8          恒顺醋业  1271.0     欧菲光  20290.0   中国平安  1026.0     欧菲光  20290.0   

       港股                                    美股                                
   1小时内最热         24小时内最热                1小时内最热              24小时内最热           
       名称      热度      名称      热度            名称      热度           名称       热度  
1  小米集团-W  1184.0  小米集团-W  7912.0           拼多多  2121.0           苹果  14453.0  
2    中烟香港   363.0    腾讯控股  6692.0  油气开采ETF-SPDR  1565.0          拼多多   7573.0  
3    腾讯控股   268.0    融创中国  6646.0            苹果  1545.0         阿里巴巴   6026.0  
4    融创中国   255.0    中烟香港  6305.0          瑞幸咖啡  1231.0          AMD   5419.0  
5     IGG   235.0    中国忠旺  5545.0          阿里巴巴  1085.0          特斯拉   5109.0  
6    中国忠旺   204.0  新城发展控股  5049.0           特斯拉   955.0  Beyond Meat   4025.0  
7  新城发展控股   176.0    中国平安  3975.0            微软   883.0           高通   3444.0  
8    中国恒大   166.0     IGG  3340.0   Beyond Meat   760.0           京东   3206.0